# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [33]:
from selenium import webdriver #Criar o Navegador
from selenium.webdriver.common.by import By #localizar elementos no site
from selenium.webdriver.common.keys import Keys #Permite clicar teclas no teclado

navegador = webdriver.Chrome()

#01 - entrar no google
navegador.get("https://www.google.com/")
#02 - pesquisar a cotação do dolar
navegador.find_element(By.XPATH, 
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("Cotação Dolar")
navegador.find_element(By.XPATH, 
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_dolar = navegador.find_element(By.XPATH, 
                       '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
display(cotacao_dolar)
#04 - pegar a cotação do euro
navegador.get("https://www.google.com/")

navegador.find_element(By.XPATH, 
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("Cotação Euro")
navegador.find_element(By.XPATH, 
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_euro = navegador.find_element(By.XPATH, 
                       '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
display(cotacao_euro)
#05 - pegar a cotação do ouro
navegador.get("https://www.melhorcambio.com/ouro-hoje")


cotacao_ouro = navegador.find_element(By.XPATH, 
                       '//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(",",".")
display(cotacao_ouro)


'5.1352'

'5.33881068'

'300.66'

#### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [34]:
#06 - atualizar planilha
import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")
display(tabela)



,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [35]:
#atualizando cotação de acordo com a moeda correspondente
#dolar

tabela.loc[tabela["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)
#euro
tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)
#ouro
tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)


#atualizando preço de compra = po*co

tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]

#atualizando preco de vendas = pc*margem
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]



display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.135200,5135.148648,1.40,7189.208107
1,Carro Renault,4500.00,Euro,5.338811,24024.648060,2.00,48049.296120
2,Notebook Dell,899.99,Dólar,5.135200,4621.628648,1.70,7856.768702
3,IPhone,799.00,Dólar,5.135200,4103.024800,1.70,6975.142160
4,Carro Fiat,3000.00,Euro,5.338811,16016.432040,1.90,30431.220876
5,Celular Xiaomi,480.48,Dólar,5.135200,2467.360896,2.00,4934.721792
6,Joia 20g,20.00,Ouro,300.660000,6013.200000,1.15,6915.180000


### Agora vamos exportar a nova base de preços atualizada

In [36]:
tabela.to_excel("Produtos_Novos.xlsx", index=False)
navegador.quit()